# Impedance Calibration Test Run

### Overview:
1. Network Preperation
1. Import Matched Trace Data
2. Specify Calibration Parameters
    - Link Impedance Function
    - Turn Impedance Function
    - Objective Function
        - Exact Overlap
        - Buffer Overlap (in progress)
        - Frechet Distance (in progress)
3. Run Calibration (in progress)
4. Export Results to Examine

In [1]:
from pathlib import Path
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
import networkx as nx
from stochopy.optimize import minimize
import stochastic_optimization
from tqdm import tqdm
import similaritymeasures
import random

from shapely.ops import LineString, MultiLineString

import sys
sys.path.insert(0,str(Path.cwd().parent))
from network.src import modeling_turns
import speedfactor

In [2]:
import json
config = json.load((Path.cwd().parent / 'config.json').open('rb'))
calibration_fp = Path(config['project_directory']) / 'Calibration'
cycleatl_fp = Path(config['project_directory']) / 'CycleAtlanta'
matching_fp = Path(config['project_directory']) / 'Map_Matching'
network_fp = Path(config['project_directory']) / 'Network'
if calibration_fp.exists() == False:
    calibration_fp.mkdir()

# Network Preperation

In [3]:
turns = pd.read_parquet(network_fp/'turns_df.parquet')
links = gpd.read_file(network_fp/'final_network.gpkg',layer='edges')
turn_G = modeling_turns.make_turn_graph(turns)

In [4]:
geo_dict = dict(zip(links['linkid'],links['geometry']))

In [5]:
#added a major/minor classification, everything else is just left as "road"
major_road = ['primary','secondary']
major_road = major_road + [item + '_link' for item in major_road]
minor_road = ['tertiary','unclassified','residential','service','trunk','living_street']
major_road = major_road + [item + '_link' for item in minor_road]
links.loc[links['highway'].isin(major_road),'link_type_new'] = 'major_road'
links.loc[links['highway'].isin(minor_road),'link_type_new'] = 'minor_road'
links.loc[links['link_type_new'].isna(),'link_type_new'] = links.loc[links['link_type_new'].isna(),'link_type']

In [6]:
#links['link_type_new'].unique()
links['high_traffic_stress'] = links['link_type_new'] == 'major_road'

Format variables (in progress)

In [7]:
#links['high_traffic_stress'] = links['highway'] == 'primary'
# df_edges['bike_facility_type'].value_counts()
# df_edges['high_traffic_stress'] = df_edges['bike_facility_type'].isna() & (df_edges['highway'].map(levels) > 4 | df_edges['speed limit'] > 30)

Format turn variables (in progress)

In [8]:
turns['left'] = turns['turn_type'] == 'left'
turns['right'] = turns['turn_type'] == 'right'

In [9]:
speedfactor.calculate_adjusted_speed(links,9)

,A,B,linkid,link_type,osmid,timestamp,version,type,highway,oneway,...,descent_grade_%,length_ft,geometry,link_type_new,high_traffic_stress,length_m,slope_factor,adjusted_speed_kph,travel_time_min,adj_travel_time_min
0,6710023987,69428484,1125582092,road,50346065,1608716546,19,way,primary,False,...,0.96,1527.030230,"LINESTRING (2197510.343 1317981.931, 2197872.9...",major_road,True,465.557997,1.004133,1.936532,1.928562,1.928562
1,6715039076,69359601,1125582111,road,630014235,1565939813,2,way,residential,False,...,0.73,256.320965,"LINESTRING (2199244.073 1317966.371, 2199301.4...",minor_road,False,78.146636,1.000590,0.323911,0.323720,0.323720
2,6715039076,69359606,1125582112,road,630014235,1565939813,2,way,residential,False,...,1.08,253.291057,"LINESTRING (2199301.414 1317716.546, 2199345.5...",minor_road,False,77.222883,1.000000,0.319894,0.319894,0.319894
3,8317885809,69302632,1125582120,service,894845300,1610524685,1,way,service,False,...,0.00,79.006478,"LINESTRING (2200338.523 1317313.398, 2200260.0...",minor_road,False,24.087341,1.002500,0.100031,0.099781,0.099781
4,8317885804,69302632,1125582121,road,9250903,1610524685,3,way,service,False,...,0.79,660.819821,"LINESTRING (2200337.281 1317974.107, 2200336.5...",minor_road,False,201.469458,1.009437,0.842458,0.834582,0.834582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178480,8754434429,8754434428,1125780316,sidewalk_or_crossing,945644551,1621641908,1,way,footway,False,...,0.00,5.766995,"LINESTRING (2223452.223 1391194.507, 2223456.8...",sidewalk_or_crossing,False,1.758230,1.031380,0.007512,0.007283,0.007283
178481,8754434425,5411749582,1125780317,road,361222946,1634991173,9,way,primary,False,...,0.45,73.539097,"LINESTRING (2223419.333 1391194.220, 2223419.7...",major_road,True,22.420456,1.000247,0.092899,0.092876,0.092876
178482,8754434431,8754434429,1125780318,bike,945644553,1621641908,1,way,cycleway,False,...,0.00,11.352379,"LINESTRING (2223456.848 1391197.953, 2223466.1...",bike,False,3.461091,1.420645,0.020368,0.014337,0.014337
178483,8754434431,5411749582,1125780319,road,561187726,1621641908,3,way,primary_link,True,...,0.00,79.665935,"LINESTRING (2223466.186 1391204.408, 2223441.9...",major_road,True,24.288395,1.004900,0.101107,0.100614,0.100614


# Specify Link Impedance Functions

## BicyclingPlus Demo Impedance Functions

Turn Only Impedance

In [10]:
#TODO allow for certain impedance functions to be left out

In [11]:
#have position of beta next to name of variable
betas_links = {} 
betas_turns = {
    0 : 'right',
    1 : 'left',
}

'''
Currently works with binary and numeric variables. Categoricals will have to be
cast into a different format for now.

Link impedance is weighted by the length of the link, turns are just the impedance associated
'''

#customize this function to change impedance formula
#TODO streamline process of trying out new impedance functions
def link_impedance_function(betas,beta_links,links):
    #prevent mutating the original links gdf
    links = links.copy()
    
    links['link_cost'] = links['adj_travel_time_min']
    
    return links

def turn_impedance_function(betas,beta_turns,turns):
    #use beta coefficient to calculate turn cost
    #base_turn_cost = 30 # from Lowry et al 2016 DOI: http://dx.doi.org/10.1016/j.tra.2016.02.003
    # turn_costs = {
    #     'left': betas[1] * base_turn_cost,
    #     'right': betas[1] * base_turn_cost,
    #     'straight': betas[1] * base_turn_cost
    # }
    #turns['turn_cost'] = turns['turn_type'].map(turn_costs)

    turns = turns.copy()

    turns['turn_cost'] = 0

    for key, item in beta_turns.items():
        turns['turn_cost'] = turns['turn_cost'] + (betas[key] * turns[item])

    turns['turn_cost'] = turns['turn_cost'].astype(float)

    return turns

Stree Impedance

In [12]:
# #have position of beta next to name of variable
# betas_links = {} 
# betas_turns = {
#     0 : 'right',
#     1 : 'left',
# }

# '''
# Currently works with binary and numeric variables. Categoricals will have to be
# cast into a different format for now.

# Link impedance is weighted by the length of the link, turns are just the impedance associated
# '''

# #customize this function to change impedance formula
# #TODO streamline process of trying out new impedance functions
# def link_impedance_function(betas,beta_links,links):
#     #prevent mutating the original links gdf
#     links = links.copy()
    
#     links['link_cost'] = links['adj_travel_time_min']
    
#     return links

# def turn_impedance_function(betas,beta_turns,turns):
#     #use beta coefficient to calculate turn cost
#     base_turn_cost = 30 # from Lowry et al 2016 DOI: http://dx.doi.org/10.1016/j.tra.2016.02.003
#     # turn_costs = {
#     #     'left': betas[1] * base_turn_cost,
#     #     'right': betas[1] * base_turn_cost,
#     #     'straight': betas[1] * base_turn_cost
#     # }
#     #turns['turn_cost'] = turns['turn_type'].map(turn_costs)

#     turns = turns.copy()

#     turns['turn_cost'] = 0

#     for key, item in beta_turns.items():
#         turns['turn_cost'] = turns['turn_cost'] + (betas[key] * turns[item])

#     turns['turn_cost'] = turns['turn_cost'].astype(float)

#     return turns

## Thesis Impedance Functions

In [13]:
# #have position of beta next to name of variable
# betas_links = {
#     0 : 'ascent_grade',
#     1 : 'high_traffic_stress',
# }

# betas_turns = {
#     2 : 'signalized',
#     3 : 'unsignalized',
#     4 : 'left',
# }

# '''
# Currently works with binary and numeric variables. Categoricals will have to be
# cast into a different format for now.

# Link impedance is weighted by the length of the link, turns are just the impedance associated
# '''

# #customize this function to change impedance formula
# #TODO streamline process of trying out new impedance functions
# def link_impedance_function(betas,beta_links,links):
#     #prevent mutating the original links gdf
#     links = links.copy()
    
#     links['attr_multiplier'] = 0

#     for key, item in beta_links.items():
#         links['attr_multiplier'] = links['attr_multiplier'] + (betas[key] * links[item])

#     links['link_cost'] = links['length_ft'] * (1 + links['attr_multiplier'])
    
#     return links

# def turn_impedance_function(betas,beta_turns,turns):
#     #use beta coefficient to calculate turn cost
#     base_turn_cost = 30 # from Lowry et al 2016 DOI: http://dx.doi.org/10.1016/j.tra.2016.02.003
#     # turn_costs = {
#     #     'left': betas[1] * base_turn_cost,
#     #     'right': betas[1] * base_turn_cost,
#     #     'straight': betas[1] * base_turn_cost
#     # }
#     #turns['turn_cost'] = turns['turn_type'].map(turn_costs)

#     turns = turns.copy()

#     turns['turn_cost'] = 0

#     for key, item in beta_turns.items():
#         turns['turn_cost'] = turns['turn_cost'] + (betas[key] * turns[item])

#     turns['turn_cost'] = turns['turn_cost'].astype(float)

#     return turns

# Import Training Set

In [14]:
with (calibration_fp/'test_set.pkl').open('rb') as fh:
    test_set = pickle.load(fh)
with (calibration_fp/'train_set.pkl').open('rb') as fh:
    train_set = pickle.load(fh)

In [15]:
import stochastic_optimization
from importlib import reload
reload(stochastic_optimization)

<module 'stochastic_optimization' from 'c:\\Users\\tpassmore6\\Documents\\GitHub\\BikewaySimDev\\impedance_calibration\\stochastic_optimization.py'>

In [17]:
ods = stochastic_optimization.match_results_to_ods(train_set)
test_ods = stochastic_optimization.match_results_to_ods(test_set)

# objective_function = stochastic_optimization.exact_overlap
# objective_function_args = {'length_dict':length_dict,'standardize':True}

objective_function = stochastic_optimization.buffer_overlap
objective_function_kwargs = {'geo_dict':geo_dict,'buffer_ft':50,'standardize':True}

#not really sure how to best set boundary conditions yet
num_of_coefs = len(betas_links) + len(betas_turns)
bounds = [[0, 50] for _ in range(0, num_of_coefs)]

In [18]:
past_betas = []
past_vals = []
args = (
    past_betas,
    past_vals,
    betas_links,betas_turns,
    ods,train_set,
    link_impedance_function,
    turn_impedance_function,
    links,turns,turn_G,
    objective_function,
    objective_function_kwargs
)

In [20]:
from importlib import reload
reload(stochastic_optimization)
reload(speedfactor)
start = time.time()
# args = (df_edges,turns,turn_G,matched_traces,False)
x = minimize(stochastic_optimization.impedance_calibration, bounds=bounds, args=args, method='pso', options={'maxiter':5})
end = time.time()
print(f'Took {(end-start)/60/60} hours')

[ 0.4 11. ]
-0.26
[17.  20.3]
-0.24
[ 5.2 35.4]
-0.24
[21.5 27.3]
-0.23
[10.2  5.6]
-0.24
[45.8 40.9]
-0.22
[26.2 46. ]
-0.22
[40.3 32.1]
-0.22
[32.1 16.4]
-0.21
[37.   1.4]
-0.24
[ 0.4 11. ]
-0.26
[ 1.5 19.1]
-0.25
[ 2.2 34.8]
-0.24
[7.3 4.8]
-0.26
[7.3 5.1]
-0.26
Negative Impedance Coefficient Detected
Negative Impedance Coefficient Detected
Negative Impedance Coefficient Detected
[29.7  6.9]
-0.21
[30.6  3.9]
-0.22
[10.6  4.7]
-0.24
Negative Impedance Coefficient Detected
Negative Impedance Coefficient Detected
Negative Impedance Coefficient Detected
[5.1 4.7]
-0.27
Negative Impedance Coefficient Detected
Negative Impedance Coefficient Detected
Negative Impedance Coefficient Detected
Negative Impedance Coefficient Detected
[12.6  5.9]
-0.23
[3.2 7.4]
-0.26
[ 9.5 14.3]
-0.25
[ 5.7 33.8]
-0.23
[10.6  4.1]
-0.24
[3.5 4.4]
-0.27
[59.6 36.1]
-0.2
[20.8 26.7]
-0.23
[47.9 22.9]
-0.21
[2.6 0.5]
-0.28
Negative Impedance Coefficient Detected
Negative Impedance Coefficient Detected
[18.4  4.8]

In [21]:
np.array(past_vals).min()

-0.28

In [27]:
past_betas[np.array(past_vals).argmin()]

(2.6, 0.5)

In [22]:
x

     fun: -0.28
 message: 'maximum number of iterations is reached'
    nfev: 50
     nit: 5
  status: -1
 success: False
       x: array([2.07883031, 1.44731444])

Create GIFs

In [23]:
import geopandas as gpd
import matplotlib.pyplot as plt
import imageio
from io import BytesIO

# Function to plot a GeoSeries and save the plot
def plot_geoseries(geoseries,other_geoseries,i,past_val):
    fig, ax = plt.subplots(figsize=(20, 20))
    #cx.add_basemap(ax)
    other_geoseries.plot(ax=ax,color='blue',weight=1)
    geoseries.plot(ax=ax,color='red')
    ax.set_title(f"Iter:{i} Overlap Function:{past_val}")
    ax.set_axis_off()
    img_bytes = BytesIO()
    plt.savefig(img_bytes, format='png', bbox_inches='tight')
    plt.close()
    return img_bytes.getvalue()

In [24]:
# num_trips = 20

# for z in range(0,num_trips):

#     #choose a random tripid
#     tripid = random.choice(list(train_set.keys()))
#     start_node = train_set[tripid]['start_node']
#     end_node = train_set[tripid]['end_node']

#     matched_edges = train_set[tripid]['matched_edges']
#     matched_edges = np.array(matched_edges)
#     matched_line = MultiLineString([geo_dict[linkid] for linkid, reverse_link in matched_edges])
#     matched_line = gpd.GeoSeries(matched_line,crs='epsg:2240')
#     matched_line = matched_line.to_crs('epsg:4326')

#     modeled_lines = []

#     for betas in past_betas:
#         #update network with the correct impedances
#         stochastic_optimization.impedance_update(betas,betas_links,betas_turns,
#                                 link_impedance_function,
#                                 turn_impedance_function,
#                                 links,turns,turn_G)
#         #find shortest path
#         modeled_edges = stochastic_optimization.impedance_path(turns,turn_G,start_node,end_node)['edge_list']
#         modeled_line = MultiLineString([geo_dict[linkid] for linkid, reverse_link in modeled_edges])
#         modeled_line = gpd.GeoSeries(modeled_line,crs='epsg:2240')
#         modeled_line = modeled_line.to_crs('epsg:4326')
#         modeled_lines.append(modeled_line)

#     # List of GeoSeries (Replace this with your own GeoSeries list)
#     geoseries_list = modeled_lines

#     # Loop through the list of GeoSeries, plot each one, and save the plot
#     images = []
#     for i, geoseries in enumerate(geoseries_list):
#         past_val = past_vals[i]
#         image_bytes = plot_geoseries(geoseries,matched_line,i,past_val)
#         images.append(imageio.imread(BytesIO(image_bytes)))

#     # Path for saving the GIF
#     gif_path = f"animations/turn_calibrate_{z}.gif"

#     # Save the images as a GIF
#     imageio.mimsave(Path.cwd()/gif_path, images, format='gif', duration=2)


#### Calculate overlap for test set
Need to use best set of betas to 

In [25]:
#link_impedance_col = "adj_travel_time_min"

#update impedances
betas = x.x
stochastic_optimization.impedance_update(betas,betas_links,betas_turns,
                          link_impedance_function,
                          turn_impedance_function,
                          links,turns,turn_G)

#find shortest path
results_dict = {(start_node,end_node):stochastic_optimization.impedance_path(turns,turn_G,start_node,end_node) for start_node, end_node in test_ods}

#calulate objective function
val_to_minimize = objective_function(test_set,results_dict,**objective_function_kwargs)
print('Test set overlap:',np.abs(np.round(val_to_minimize,2)))

-0.28

## Visualize random trip

In [26]:
# #retrieve chosen path linkids and convert them to tuple
# chosen = [tuple(row) for row in match_results[tripid]['matched_edges'].to_numpy()]
# shortest = [tuple(row) for row in match_results[tripid]['shortest_edges'].to_numpy()]
# #retrieve modeled path linkids
# modeled_edges = results_dict[(start_node,end_node)]['edge_list']

# #get geos (non-directional)
# chosen_geo = [geo_dict[linkid[0]] for linkid in chosen]
# shortest_geo = [geo_dict[linkid[0]] for linkid in shortest]
# modeled_geo = [geo_dict[linkid[0]] for linkid in modeled_edges]

# chosen_lines = gpd.GeoSeries(chosen_geo,crs='epsg:2240')
# shortest_lines = gpd.GeoSeries(shortest_geo,crs='epsg:2240')
# modeled_lines = gpd.GeoSeries(modeled_geo,crs='epsg:2240')

# stochastic_optimization.visualize_three_no_legend(chosen_lines,shortest_lines,modeled_lines)